# A moving surface problem:
In this example we consider a moving surface problem:

$$
\dot{u} + ({div}_\Gamma \mathbf{w} ) u - \nu \Delta_\Gamma u = 0 \quad on \quad \Gamma(t),   \qquad  t \in [0,T]   
$$
where $\Gamma(t)$ changes in time (while $\Gamma(t)$ is always smooth) and is described by a level set function $\phi(t)$.

We follow an approach from [1] where we extend the solution u into a small neighborhood $\mathcal{O}(\Gamma(t))$ and there solve the related problem
\begin{align}
  \partial_t u + \mathbf{w} \cdot \nabla u + ({div}_\Gamma \mathbf{w} ) u - \nu \Delta_\Gamma u &= 0 \quad \text{on} \quad \Gamma(t) \\
  \nabla u \cdot \nabla \phi & = 0 \quad \text{in} \quad \mathcal{O}(\Gamma(t)).
\end{align}
The second equation determines u to be a constant extension in normal direction of the solution on the surface. This extension allows to evaluate u at an older time instance on the surface which may have moved as long as the neighborhood is sufficiently large. Hence, we require $\Gamma_h^n \subset \mathcal{O}(\Gamma_h^{n-1})$ where $\Gamma_h^k,~k=n-1,n$ are the discrete surface approximations of two subsequent time steps.

This allows to apply a [method of lines](http://www.scholarpedia.org/article/Method_of_lines) approach. Here, we consider the backward Euler method:

\begin{align}
  \frac{u^n - u^{n-1}}{\Delta t} + \mathbf{w}^n \cdot \nabla u^n + ({div}_\Gamma \mathbf{w}^n ) u^n - \nu \Delta_\Gamma u^n &= 0 \quad \text{on} \quad \Gamma^n \\
  \nabla u^n \cdot \nabla \phi^n & = 0 \quad \text{in} \quad \mathcal{O}(\Gamma^n).
\end{align}

Note that $u^{n-1}$ is well-defined on $\Gamma^n$ as $\Gamma^n \subset \mathcal{O}(\Gamma^{n-1})$.

The realization of the extension is included in each time step solves as a stabilization. This simplifies the realization of the extension, but also allows to obtain reasonable condition number bounds. Let $V_h^n$ be the finite element space to the part of the mesh that is touched by $\mathcal{0}(\Gamma^n)$, then every time step is discretized with (for $u^{n-1} \in V_h^{n-1} \subset L^2(\mathcal{O}(\Gamma^{n-1})$):

Find $u_h^n \in V_h^n$, s.t.
\begin{align}
  \int_{\Gamma_h^n} \left\{ \frac{u_h^n -u_h^{n-1}}{\Delta t} v_h + \frac12 (w_T^e \cdot \nabla_{\Gamma_h} u_h^n v_h - \mathbf{w}_T^e \cdot \nabla_{\Gamma_h} v_h u_h^n) + ({div}_{\Gamma_h} (\mathbf{w}^e -\frac12 \mathbf{w}_T^e) u_h^n v_h \right \} ds_h &  \\
  + \nu \int_{\Gamma_h^n} \nabla_{\Gamma_h} u_h^n \cdot \nabla_{\Gamma_h} v_h ds_h + \rho_n \int_{\mathcal{O}(\Gamma_h^n)} (\mathbf{n}_h \cdot \nabla u_h^n) (\mathbf{n}_h \cdot \nabla v_h^n) d \mathbf{x} & = 0
\end{align}
for all $v_h \in V_h^n$.

Literature:
----------------------
[1]: C. Lehrenfeld, M.A. Olshanskii, and X. Xu. A stabilized trace finite element method for partial differential equations on evolving surfaces. arXiv preprint arXiv:1709.07117, 2017. [ [http](https://arxiv.org/abs/1709.07117) | [.pdf](https://arxiv.org/pdf/1709.05832.pdf) ]


In [ ]:
import netgen.gui
%gui tk
import tkinter

from ngsolve import *
from ngsolve.comp import *
from ngsolve.utils import *
from ngsolve.internal import *
from xfem import *
from netgen.geom2d import SplineGeometry
from xfem.lsetcurv import *
from math import pi
from time import sleep

# Building blocks of the method

## Description of the geometry

We use a levelset function $\phi(t) =  \sqrt{x^2 + (y-\rho(t))^2} - r_{0}$ with
$r_{0} = 1/2$, to describe the evolving geometry. The moving domain is then given as the set of points where 
the levelset function takes negative values:
 $ \Omega(t) = \{ (x,y) \in \mathbb{R}^2 \mid \phi(x,y,t) < 0 \}$.
The time-dependent domain $\Omega(t)$ is contained in a larger, time-independent domain $\Omega = [-1,1] \times [-0.75,1.5]$.
The mesh is contructed on the background domain $\Omega$ and unfitted to $\Omega(t)$.

In [ ]:
# geometry
square = SplineGeometry()
square.AddRectangle([-1,-0.75],[1,1.5],bc=1)
maxh = 0.1
mesh = Mesh (square.GenerateMesh(maxh=maxh))

# keeping track of time
t = Parameter(0)
tnew = 0
tend = 0.5
delta_t = 0.01

# the levelset function that describes the domain's evolution
r0 = 0.5
rho =  CoefficientFunction((1/(pi))*sin(2*pi*t))
levelset= CoefficientFunction(sqrt(x*x+(y-rho)*(y-rho)) -r0)

# mesh adaptation class that also calculates a P1 approximation to a levelset function
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=1, threshold=0.1, discontinuous_qn=True)
# visualize the domain's evolution
while tend - tnew > delta_t/2:
    if netgen.gui.win.tk.dooneevent(tkinter._tkinter.DONT_WAIT):
        pass
    deformation = lsetmeshadap.CalcDeformation(levelset)
    Draw(IfPos(-lsetmeshadap.lset_p1,CoefficientFunction(1),CoefficientFunction(0)),mesh,"movingDom")
    tnew +=  delta_t
    t.Set(tnew)
    sleep(0.15)

## Finite Element Space for discretization on time slab

The problem will be treated by a Space-Time DG Method. In order to avoid 2+1 dimensional complexity, the Space-Time domain will be split into so called time slabs:
$Q^{n} = \{ (x,y,t) \mid (x,y) \in \Omega(t), \, t \in (t_{n-1},t_{n}] \} $. We will derive a suitable weak variational formulation on the $Q^{n}$ that allows to solve the problem time slab per time slab. This leads to the variational structure of a time stepping scheme. To this end, we need a Finite Element space on the time slab:

\begin{equation*}
W_{n} = \{ v: Q^{n} \rightarrow \mathbb{R} \mid v(x,t) = \sum_{m= 0}^{k_{t}}{ \phi_{m}(t) u_{m}(x)}, \, \varphi_{m} \in V_{h}, m=0, \ldots, k_{t}  \}.
\end{equation*}

The functions in $W_{n}$ are tensor products between basis functions in time $\phi_{m}(t)$ and purely space-dependent functions $u_{m}(x)$ from a standard Finite-Element Space $V_{h}$ of order $k_{s}$ defined on the background domain $\Omega$. The polynomial degrees in space and time can be prescribed independently.

## Python file 

A corresponding Python file for the method is available from http://www.github.com/ngsxfem/ngsxfem.